In [2]:
import cv2
import os
from time import time
import glob
import h5py
import pandas as pd

In [3]:
os.chdir("/home/donghan/DeepLabCut/data/")
filenames = glob.glob('*.h5') 
#Return the file name with extention of .h5, which contain the data of coordination axis
f = []
for filename in filenames:
    f = h5py.File(filename, 'r')
    start = filename.find('10') 
    #Find the string that start with "10"
    end = filename.find(' rotated', start) 
    #Return the string with end of " rotated", aims to name the file
    csvfile = []
    with pd.HDFStore(filename, 'r') as d:
        df = d.get(list(f.keys())[0])
        df.to_csv(filename[start:end] + '.csv') 
        #Automaticaly change to unique file name with specific mouse number
        csvfile.append(filename[start:end] + '.csv')
for i in csvfile:
    data = pd.read_csv(i, skiprows = 2) 
    #Skip the rows of scorer and bodyparts
    move_data = data.loc[260:] 

In [5]:
def videoWriter(task, filename, outputName, corrX, corrY, distX, distY, displayVideo = False):
    
    '''
    Task: Takes video as input and returns its capturing and output file. 
    
    If task is edited, then corrX and corrY is the vertex of the cropped video edge. 
    '''

    # Capture video
    cap = cv2.VideoCapture(filename)

    # Read video frame by frame
    # Extract original video frame features
    sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # Make a directory to store the processed videos
    path = "./" + task
    try:  
        os.mkdir(path)
        print ("Successfully created the directory %s " % path)
    except OSError:  
        pass
    

    #Automatically name the processed videos  
    file = "./" + task + "/" + outputName
    if task == "edited":
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (distY, distX)) 
        # Another option: cv2.VideoWriter_fourcc(*'XVID')
    else:
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, sz)

    return cap,out
    
def videoEditer(filename, outputName, corrX, corrY, distX, distY, displayVideo = False):

    '''
    Task: Crop videos to actural mouse moving edge
    
    PARAMETERS:
    video: file that await for writting
    
    filename: video that need to be trim
    
    outputName: filename of output video
    
    corrX: coordinates of the start point in X
    
    corrY: coordinates of the start point in Y
    
    dist: cropped video is a square, as long as the coordinates of the start point is defined, the other
    three vertex points are clear. 

    '''
    # Calculate the other three points' coordinate
    corrX1 = corrX + distX
    corrY1 = corrY + distY

    video = videoWriter("edited", filename, outputName, corrX, corrY, distX, distY, displayVideo)
    cap = video[0] 
    out = video[1]                  
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    # Not capturing a video
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 1
    # Read videos and rotate by certain degrees
    while(cap.isOpened()):
        # Flip for truning(fliping) frames of video
        ret,img = cap.read()
        try:
            img2 = img[corrX:corrX1, corrY:corrY1]
            out.write(img2)
            if displayVideo == True:
                cv2.imshow('edited video',img2) 
            if count == frameCnt:
                print (filename, 'successfully ', "edited!")
                break
            else:
                count += 1
            k=cv2.waitKey(30) & 0xff
            #once you inter Esc capturing will stop
            if k==27:
                break
        except:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    

    
def getCoord(data):
    '''
    Task: Obtaining the coordinates of cropped points
    '''
    xMax = max((data['x']).astype(int))
    xMin = min((data['x']).astype(int))
    yMax = max((data['y']).astype(int))
    yMin = min((data['y']).astype(int))
    
    lengthX = xMax - xMin
    lengthY = yMax - yMin
    
#     if yMax - yMin < xMax - xMin:
#         length = yMax - yMin
#     else:
#         length = xMax - xMin
    
    
    return xMin, yMin, lengthX, lengthY



In [48]:
videoEditer('1034 SI_B, Aug 15, 13 7 49.mp4', "cropTest.mp4", 10,10,300)

1034 SI_B, Aug 15, 13 7 49.mp4 successfully  edited!


In [6]:
x, y, lengthX, lengthY = getCoord(move_data)

In [74]:
sorted(move_data['y'], reverse = True)

[360.3071541786194,
 359.8821659088135,
 359.31144404411316,
 358.87915778160095,
 358.1923756599426,
 358.1311404705048,
 358.06207633018494,
 357.9358524084091,
 357.7606500387192,
 357.6735737323761,
 357.59456861019135,
 355.6365961134434,
 355.54499647021294,
 354.61147129535675,
 354.470511674881,
 354.43358445167536,
 354.16579627990717,
 354.15162658691406,
 354.05973434448237,
 353.7722930908203,
 353.0516414642334,
 352.6844549179077,
 352.4438591003418,
 352.3019428253174,
 352.2593936920166,
 352.1821346282959,
 352.02808380126953,
 352.0044822692871,
 351.8965072631836,
 351.8884267807007,
 351.53300547599787,
 351.37711906433105,
 351.33116722106934,
 351.1972632408142,
 351.11696338653564,
 351.0705556869507,
 350.64591789245605,
 350.17234802246094,
 350.16414737701416,
 350.0765838623047,
 349.74489307403564,
 349.68533611297613,
 349.6158981323242,
 349.52703189849854,
 349.483097076416,
 349.4533348083496,
 349.4153289794922,
 349.07039642333984,
 349.0191488265991,


In [8]:
move_data

coords           x           y  likelihood         x.1         y.1  \
260      260  262.273312   88.680502    0.897856  295.308178  114.930924   
261      261  257.752191   89.590469    0.901112  293.086969  115.709660   
262      262  255.663816   97.471930    0.825897  291.239094  115.826995   
263      263  244.900141  109.774375    0.939061  292.035050  114.781725   
264      264  238.525290  112.144867    0.949862  293.139378  109.262915   
265      265  230.563351  112.675102    0.869110  290.275887  106.801480   
266      266  221.549237  114.933034    0.929303  272.347570  102.884142   
267      267  212.595529  119.447464    0.952777  261.571139  101.835020   
268      268  207.410507  120.173340    0.920020  246.553816  114.353611   
269      269  195.802108  121.963829    0.965004  242.046974  111.888453   
270      270  191.456611  121.205399    0.935960  240.593262  104.800985   
271      271  179.650813  121.556475    0.963086  225.102430  106.559156   
272      272  174.439103  123.479878    0.878629  215.475766  116.773801   
273      273  160.916546  128.047458    0.883628  212.010269  115.761955   
274      274  153.178155  134.418221    0.862310  198.610745  119.041553   
275      275  147.385378  135.256189    0.957252  194.591497  119.754597   
276      276  143.093574  135.539277    0.947605  187.561649  118.402503   
277      277  136.703259  141.835145    0.906575  180.326770  115.829439   
278      278  132.361131  142.704658    0.921992  174.389800  115.545342   
279      279  129.026486  143.850581    0.838082  170.830076  122.040218   
280      280  124.265150  149.117263    0.771991  166.736177  130.679912   
281      281  120.169991  158.091252    0.951077  162.409517  127.107612   
282      282  120.807496  158.668427    0.956823  158.620220  127.288561   
283      283  120.704447  158.109483    0.956633  156.831905  118.147749   
284      284  121.162522  158.524552    0.946151  158.271128  117.797369   
285      285  123.609108  158.195817    0.869138  157.474808  119.417065   
286      286  120.465477  161.401819    0.963375  157.427969  117.442783   
287      287  118.320897  159.820478    0.955488  157.589911  117.377086   
288      288  116.177217  158.864885    0.945621  157.693790  118.534067   
289      289  114.256661  159.980041    0.951815  156.888603  118.896014   
...      ...         ...         ...         ...         ...         ...   
3451    3451  150.839350  121.584011    0.896112  143.341560  168.242155   
3452    3452  155.237265  116.495194    0.905704  133.736922  159.899997   
3453    3453  154.633912  101.778252    0.943964  132.640466  149.717928   
3454    3454  151.695936   89.496385    0.943384  143.575905  135.425034   
3455    3455  152.969257   83.659161    0.974177  147.956470  128.320521   
3456    3456  149.707090   76.245843    0.927018  143.685810  123.455411   
3457    3457  148.629270   69.905497    0.951913  123.848154  117.051071   
3458    3458  144.231025   61.845531    0.937684  126.220215  108.717028   
3459    3459  145.423819   52.809326    0.956221  124.308053  102.493972   
3460    3460  148.270969   41.311861    0.930796  131.313589   94.466664   
3461    3461  149.707768   42.049118    0.945051  117.406098   87.502764   
3462    3462  155.584792   50.999450    0.838977  116.807371   86.113835   
3463    3463  159.665683   54.729313    0.917486  114.644156   81.530432   
3464    3464  170.005055   56.981374    0.936175  117.396525   71.089319   
3465    3465  177.129623   53.667091    0.941148  125.024396   68.628499   
3466    3466  180.512795   53.774270    0.964735  130.490648   65.377372   
3467    3467  188.832328   52.318870    0.907640  130.891912   54.181355   
3468    3468  190.728322   47.382530    0.921018  133.090011   45.028707   
3469    3469  202.136685   43.091707    0.815948  147.287050   44.479336   
3470    3470  207.844775   40.816784    0.941613  151.643853   44.491158   
3471    3471  213.558818   42.892617    0.945233  162.

In [83]:
x, y, lengthX, lengthY

(52, 34, 338, 326)

In [85]:
videoEditer('1034 SI_B, Aug 15, 13 7 49.mp4', "cropTest.mp4", x, y, lengthX, lengthY)

1034 SI_B, Aug 15, 13 7 49.mp4 successfully  edited!


In [21]:
head = list(zip(round(move_data['x']).astype(int), round(move_data['y']).astype(int)))
tail = list(zip(round(data['x.1']).astype(int), round(data['y.1']).astype(int)))

In [7]:
p = cv2.imread("test.jpg")
cv2.imwrite("test1.jpg",p[0:100,0:200])

True

In [23]:

sorted([i[0] for i in head], reverse = True)

[435,
 424,
 390,
 389,
 389,
 389,
 388,
 387,
 387,
 387,
 387,
 386,
 386,
 386,
 385,
 385,
 385,
 385,
 384,
 384,
 384,
 382,
 382,
 382,
 382,
 381,
 380,
 380,
 379,
 379,
 379,
 379,
 378,
 378,
 378,
 378,
 378,
 377,
 377,
 377,
 377,
 376,
 376,
 376,
 376,
 376,
 375,
 375,
 375,
 375,
 375,
 375,
 375,
 375,
 374,
 374,
 374,
 374,
 374,
 374,
 374,
 374,
 374,
 373,
 373,
 373,
 373,
 373,
 373,
 372,
 372,
 372,
 372,
 372,
 372,
 372,
 372,
 372,
 372,
 372,
 372,
 372,
 371,
 371,
 371,
 371,
 371,
 371,
 371,
 371,
 371,
 371,
 370,
 370,
 370,
 370,
 370,
 370,
 370,
 370,
 370,
 370,
 370,
 370,
 370,
 369,
 369,
 369,
 369,
 369,
 369,
 369,
 369,
 369,
 369,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 368,
 367,
 367,
 367,
 367,
 367,
 367,
 367,
 367,
 367,
 367,
 367,
 366,
 366,
 366,
 366,
 366,
 366,
 366,
 366,
 366,
 365,
 365,
 365,
 365,
 365,
 364,
 364,
 364,
 364,
 364,
 364

In [15]:
import imutils
# load the image and compute the ratio of the old height
# to the new height, clone it, and resize it
image = cv2.imread("test2.jpg")
ratio = image.shape[0] / 500.0
orig = image.copy()
image = imutils.resize(image, height = 500)
 
# convert the image to grayscale, blur it, and find edges
# in the image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 75, 200)
 
# show the original image and the edge detected image
print("STEP 1: Edge Detection")
cv2.imshow("Image", image)
cv2.imshow("Edged", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

STEP 1: Edge Detection


In [20]:
# find the contours in the edged image, keeping only the
# largest ones, and initialize the screen contour
cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
 
# loop over the contours
for c in cnts:
    # approximate the contour
    peri = cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, 0.02 * peri, True)
 
    # if our approximated contour has four points, then we
    # can assume that we have found our screen
    if len(approx) == 4:
        screenCnt = approx
        break
 
 # show the contour (outline) of the piece of paper
print("STEP 2: Find contours of paper")
cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
cv2.imshow("Outline", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

STEP 2: Find contours of paper


In [23]:
def four_point_transform(image, pts):
    # obtain a consistent order of the points and unpack them
    # individually
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    # compute the width of the new image, which will be the
    # maximum distance between bottom-right and bottom-left
    # x-coordiates or the top-right and top-left x-coordinates
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # compute the height of the new image, which will be the
    # maximum distance between the top-right and bottom-right
    # y-coordinates or the top-left and bottom-left y-coordinates
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # now that we have the dimensions of the new image, construct
    # the set of destination points to obtain a "birds eye view",
    # (i.e. top-down view) of the image, again specifying points
    # in the top-left, top-right, bottom-right, and bottom-left
    # order
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")

    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    # return the warped image
    return warped

In [25]:
# apply the four point transform to obtain a top-down
# view of the original image
warped = four_point_transform(orig, screenCnt.reshape(4, 1) * ratio)
 
# convert the warped image to grayscale, then threshold it
# to give it that 'black and white' paper effect
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
T = threshold_local(warped, 11, offset = 10, method = "gaussian")
warped = (warped > T).astype("uint8") * 255
 
# show the original and scanned images
print("STEP 3: Apply perspective transform")
cv2.imshow("Original", imutils.resize(orig, height = 650))
cv2.imshow("Scanned", imutils.resize(warped, height = 650))
cv2.waitKey(0)

NameError: name 'order_points' is not defined

In [5]:
video = videoWriter("crop", '1034 SI_B, Aug 15, 13 7 49.mp4', "cropTest.mp4")
cap = video[0]
out = video[1]


Successfully created the directory ./crop 
